In [1]:
print(sc.uiWebUrl)

http://midway3-0172.rcc.local:4041


In [2]:
! squeue -u yjia2

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          41323339   caslake _interac    yjia2  R       6:00      1 midway3-0172


In [3]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = "/home/yjia2/fonts/NotoSansCJKsc-Regular.otf"  
fm.fontManager.addfont(font_path)
zh_font = fm.FontProperties(fname=font_path)

plt.rcParams["font.family"] = zh_font.get_name()
plt.rcParams["axes.unicode_minus"] = False

from pyspark.sql import functions as F, types as T
from pyspark.ml.functions import vector_to_array
from pyspark.storagelevel import StorageLevel
from pyspark.ml.feature import Word2VecModel
from mpl_toolkits.mplot3d import Axes3D
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA

In [7]:
# Pre-processing

BASE_PATH = "cleaned_segmented_parquet"

df_raw = spark.read.parquet(BASE_PATH)

df = (
    df_raw
    .withColumnRenamed("文本内容_清洗", "content_cleaned")
    .withColumnRenamed("年份", "year")
    .select("content_cleaned", "year")
)

df = df.withColumn("year", F.col("year").cast("int"))

df = (
    df
    .withColumn("content_cleaned", F.trim(F.col("content_cleaned")))
    .filter(
        F.col("content_cleaned").isNotNull()
        & (F.length("content_cleaned") > 0)
    )
)

df = (
    df
    .withColumn("tokens", F.split(F.col("content_cleaned"), r"\s+"))
    .withColumn("tokens", F.expr("filter(tokens, x -> x <> '')"))
)

df = (
    df
    .withColumn("len_tokens", F.size("tokens"))
    .filter(F.col("len_tokens") >= 3)
    .drop("len_tokens")
)

year_stats = (
    df.groupBy("year")
      .agg(
          F.count("*").alias("n_docs"),
          F.sum(F.size("tokens")).alias("n_tokens")
      )
      .orderBy("year")
)

year_stats.show(1, truncate=False)


[Stage 14:=====================================================>(248 + 3) / 251]

+----+------+--------+
|year|n_docs|n_tokens|
+----+------+--------+
|1947|10769 |2711874 |
+----+------+--------+
only showing top 1 row



In [8]:
# Word2Vec hyper-parameters 
W2V_PARAMS = dict(
    vectorSize=150,
    windowSize=6,
    minCount=30,
    maxIter=3,          # epochs
    stepSize=0.025,     # learning rate
    numPartitions=100,
    maxSentenceLength=600,
    seed=42,
    inputCol="tokens",
    outputCol="w2v_features"
)

BASE_MODEL_DIR = "w2v_models/pd_w2v_vs150_mc30_ep3_ws6_by_year"

In [9]:
years = (
    df.select("year")
      .distinct()
      .orderBy("year")
      .rdd.map(lambda r: r["year"])
      .collect()
)

print("Years to train:", years[:10], "... total =", len(years))

[Stage 24:>                                                         (0 + 1) / 1]

Years to train: [1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956] ... total = 78


In [10]:
MIN_DOCS = 500

for y in years:
    print(f"\n===== Training year {y} =====")

    df_y = (
        df.filter(F.col("year") == y)
          .select("tokens")
    )

    n_docs = df_y.count()
    print(f"Docs in {y} = {n_docs}")

    if n_docs < MIN_DOCS:
        print(f"Skip {y} (n_docs < MIN_DOCS={MIN_DOCS})")
        continue

    df_y_train = df_y.repartition(100).persist(StorageLevel.MEMORY_AND_DISK)
    _ = df_y_train.count()   
    print(f"{y}: cached training corpus")

    w2v = Word2Vec(**W2V_PARAMS)
    w2v_model = w2v.fit(df_y_train)
    print(f"{y}: model trained")

    df_y_train.unpersist()

    model_dir = f"{BASE_MODEL_DIR}/year={y}"
    print(f"{y}: saving model to {model_dir}")
    w2v_model.write().overwrite().save(model_dir)

    vecs = w2v_model.getVectors()   # DataFrame[word: string, vector: vector]
    vecs.write.mode("overwrite").parquet(f"{model_dir}/vectors_parquet")

    vocab_size = vecs.count()
    print(f"{y}: vocab_size = {vocab_size}")

print("\nAll yearly trainings done.")


===== Training year 1947 =====
Docs in 1947 = 10769


1947: cached training corpus


1947: model trained
1947: saving model to w2v_models/pd_w2v_vs150_mc30_ep3_ws6_by_year/year=1947


1947: vocab_size = 10630

===== Training year 1948 =====


Docs in 1948 = 8246


...

2023: cached training corpus


2023: model trained
2023: saving model to w2v_models/pd_w2v_vs150_mc30_ep3_ws6_by_year/year=2023
25/11/13 17:47:42 WARN TaskSetManager: Stage 2408 contains a task of very large size (1061 KiB). The maximum recommended task size is 1000 KiB.
2023: vocab_size = 26018

===== Training year 2024 =====


Docs in 2024 = 17096


2024: cached training corpus


2024: model trained
2024: saving model to w2v_models/pd_w2v_vs150_mc30_ep3_ws6_by_year/year=2024
2024: vocab_size = 19852

All yearly trainings done.


In [12]:
from pyspark.ml.feature import Word2VecModel

BASE_MODEL_DIR = "w2v_models/pd_w2v_vs150_mc30_ep3_ws6_by_year"

def show_neighbors_for_year(year: int, word: str, topn: int = 20):
    """
    Load the yearly Word2Vec model and show nearest neighbors for a given word.
    """
    model_path = f"{BASE_MODEL_DIR}/year={year}"
    print(f"\n=== Year {year} | word = {word} ===")

    # Load model
    w2v_model = Word2VecModel.load(model_path)

    # Check if the word exists first (avoid errors)
    vecs = w2v_model.getVectors()
    if vecs.filter(F.col("word") == word).limit(1).count() == 0:
        print(f"'{word}' not in vocabulary for year {year}.")
        return

    # Show nearest neighbors
    syn = w2v_model.findSynonyms(word, topn)
    syn.show(truncate=False)

# Example: check a few years
for y in [1950, 1978, 1992, 2008, 2018]:
    show_neighbors_for_year(y, "人民", topn=15)



=== Year 1950 | word = 人民 ===


+--------+-------------------+
|word    |similarity         |
+--------+-------------------+
|万众一心|0.5745235085487366 |
|正义事业|0.5567136406898499 |
|自由民主|0.5509663820266724 |
|死敌    |0.5310417413711548 |
|人民意志|0.5274190306663513 |
|正义    |0.5226737260818481 |
|各族人民|0.5216115117073059 |
|英勇斗争|0.5147422552108765 |
|不可动摇|0.5076724290847778 |
|爱好和平|0.5045686960220337 |
|正义战争|0.5042296051979065 |
|解放事业|0.5029891133308411 |
|深信    |0.5010573863983154 |
|坚信    |0.49985629320144653|
|亿万人民|0.49916237592697144|
+--------+-------------------+


=== Year 1978 | word = 人民 ===
+--------+------------------+
|word    |similarity        |
+--------+------------------+
|正义事业|0.6518587470054626|
|同情    |0.5929450988769531|
|始终不渝|0.5833016633987427|
|解放事业|0.5722860097885132|
|珍视    |0.5614731907844543|
|声援    |0.5534377098083496|
|愿望    |0.5524501800537109|
|正义    |0.5487756133079529|
|正义斗争|0.5425110459327698|
|衷心祝愿|0.5412216186523438|
|昌盛    |0.5387455224990845|
|民族解放|0.5379835963249207|
|坚信    |0.5296506